In [1]:
import requests
from bs4 import BeautifulSoup
import re
import csv
from time import sleep
import random

In [3]:
smartlab_last_page = 'https://smart-lab.ru/forum/SBER/page5883/'

In [4]:
r = requests.get(smartlab_last_page)

In [ ]:
smartlab_html = smartlab_get_html(smartlab_last_page)

In [ ]:
smartlab_soup = BeautifulSoup(smartlab_html, 'lxml')

In [ ]:
user_agent = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 

In [2]:
def smartlab_get_html(smartlab_url):
    r = requests.get(url, headers=user_agent)
    return r.text

def smartlab_get_page_data(url, smartlab_writer):
    html = smartlab_get_html(url)
    soup = BeautifulSoup(html, 'lxml')
    comments = soup.find_all('li', {'data-type': 'comment'})
    for comment in comments:
        name = comment.find('a', class_='a_name trader_other').text
        ct = comment.find('div', class_='text')
        if 'blockquote' in str(ct):
            try:
                ct.find("blockquote").extract()
            except:
                pass
        text = ct.text.strip()
        # print(text)
        date = comment.find('time')['datetime'][:10]
        data = {'name': name,
                    'text': text,
                    'date': date,
                    'url': url}
        smartlab_writer.writerow(data)

def smartlab_get_last_page_number():
    smartlab_last_page = 'https://smart-lab.ru/forum/SBER/page5883/'
    try:
        smartlab_html = smartlab_get_html(smartlab_last_page)
        smartlab_soup = BeautifulSoup(smartlab_html, 'lxml')
        smartlab_last_page_number = smartlab_soup.find('div', {'class': 'pagination1'}).find_all('a')[-1]['href'][-5:-1]
    except:
        print('SMARTLAB - CANT GET LAST PAGE NUMBER, SOME ERROR')
        raise TypeError
    return smartlab_last_page_number

smartlab_file_name = 'smartlab_sber_downloaded.csv'

smartlab_url = 'https://smart-lab.ru/forum/SBER/page{}/'

In [5]:
# with open(smartlab_file_name, 'a') as f:
#     smartlab_order = ['name', 'text', 'date', 'url']
#     smartlab_writer = csv.DictWriter(f, fieldnames=smartlab_order)
#     for i in range(5887, 5890):
#         smartlab_get_page_data(smartlab_url.format(str(i)), smartlab_writer)
#         sleep(random.randrange(2,7))

In [7]:
def make_daily_download(smartlab_file_name):
    smartlab_last_page_number = smartlab_get_last_page_number()
    with open(smartlab_file_name, "r") as file:
        scontents = file.readlines()
        sstr1 = scontents[-1]
        #str1 = file.readline()
        smartlab_downloaded_page_number = sstr1.split()[-1][-5:-1]
    if int(smartlab_last_page_number)-2 > int(smartlab_downloaded_page_number):
        with open(smartlab_file_name, 'a') as f:
            smartlab_order = ['name', 'text', 'date', 'url']
            smartlab_writer = csv.DictWriter(f, fieldnames=smartlab_order)
            for i in range(int(smartlab_downloaded_page_number)+1, int(smartlab_last_page_number)-1): # ЗДЕСЬ ПОМЕНЯЛ
                smartlab_get_page_data(smartlab_url.format(str(i)), smartlab_writer)
                sleep(random.randrange(1,3))
        print('smartlab_writer_TEST: work done, FILE CLOSED')